In [1]:
query = "Israel Gaza War"
from article_preprocess import gather_article_metadata
from generate_database import find_articles_by_topic

/Users/allenbaranov/opt/anaconda3/envs/news/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-18 12:15:41.376406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 12:15:47.826676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Al

In [2]:
articles = find_articles_by_topic(query)

In [8]:
import re, requests
from bs4 import BeautifulSoup

def fetch_article_id(article):        
    pattern = re.compile(r'/ar-([A-Za-z0-9]+)')
    article_url = article['url']

    match = pattern.search(article_url)
    if match:
        article_id = match.group(1)
        return article_id
    else:
        raise Exception("No article ID found")



def fetch_article_contents(article_id):
    '''
    Returns article's author and contents of the article
    '''
    asset_url = "https://assets.msn.com/content/view/v2/Detail/en-us/" + article_id

    try:
        response = requests.get(asset_url)
        response.raise_for_status()
        data = response.json()
        html_content = data.get('body', 'No content found')

    except requests.RequestException as e:
        print(f"Error fetching article: {e}")
        return None
    
    print(data.get('authors', False))
    if data.get('authors', False):
        author = data.get('authors', 'None')[0]['name']
    else:
        author = 'Not found'
    
    soup = BeautifulSoup(html_content, 'lxml')
    paragraphs = [p.get_text(separator=' ', strip=True) for p in soup.find_all('p')]
    #print(author)
    return author, '\n\n'.join(paragraphs)

fetch_article_contents(fetch_article_id(articles[0]))

[{'name': 'AFP'}]


('AFP',
 'Turkish first division club Basaksekir on Thursday\xa0ended the loan spell of \xa0Israeli midfielder Eden Karzev who reposted a social media message calling for the release of hostages held by Hamas militants.\n\nThe Istanbul side launched disciplinary proceedings against Karzev earlier this week after the defensive midfielder shared a message reading: "Bring Them Home Now".\n\nThe probe came the same day Israeli footballer Sagiv Jehezkel was briefly detained and charged with incitement to hatred for displaying a message about the Gaza war after scoring a goal for Antalyaspor.\n\nJehezkel left Turkey after being formally charged in an Istanbul court on Monday.\n\nBasaksekir took Karzev on loan from Israeli outfit Maccabi Tel Aviv last year.\n\n"The player was fined in accordance with the club\'s disciplinary instructions, and it was deemed appropriate for the football player to continue his career abroad," Basaksekir said in a statement.\n\nMaccabi said the 23-year-old -- cap

In [14]:
print(gather_article_metadata(articles[1], query))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{'url': 'https://www.msn.com/en-us/news/world/inside-the-effort-to-create-a-far-reaching-us-saudi-israeli-pact-to-end-the-war/ar-AA1nbySx', 'date_published': '2024-01-18 15:06:04', 'publisher': 'NBC News', 'category': 'World', 'keywords': ['Saudi Arabia'], 'topic': 'Israel Gaza War', 'author': 'Anna Schecter', 'sentiment': 0.0002288579602133144, 'semantic_embedding': [-0.007008051034063101, -0.022199824452400208, 0.02774321287870407, -0.015907684341073036, -0.002177290618419647, 0.020768001675605774, -0.021766336634755135, -0.013135991059243679, -0.01996670663356781, -0.023171888664364815, 0.017904356122016907, 0.0027372122276574373, 0.005894775968044996, 0.0050901966

In [7]:
articles[3]

{'name': 'Qatari finance minister says Gaza war to slow Middle East economies',
 'url': 'https://www.msn.com/en-us/money/markets/qatari-finance-minister-says-gaza-war-to-slow-middle-east-economies/ar-AA1nb9MM',
 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.aaio68qySvJ6l2GA7Oi2DS&pid=News',
   'width': 700,
   'height': 466}},
 'description': "By Maha El Dahan and Dmitry Zhdannikov DAVOS (Reuters) - The war in Gaza will hit economies across the Middle East if it is not resolved and the conflict urgently needs a non-military solution, Qatar's finance minister told Reuters.",
 'provider': [{'_type': 'Organization',
   'name': 'Reuters on MSN.com',
   'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=ODF.jFXbg3L7Ce_1pS4_IOR8CA&pid=news'}}}],
 'datePublished': '2024-01-18T13:10:06.0000000Z',
 'isBreakingNews': True}

In [12]:
metadata = {}
article = articles[3]
metadata['keywords'] = [d['name'] for d in article['about']] if 'about' in article else [d['name'] for d in article['mentions']] if 'mentions' in article else "Not found"
print(metadata)

{'keywords': 'Not found'}


In [19]:
from article_preprocess import gather_article_metadata
import requests
import json
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

load_dotenv("../vars.env")

def connect_to_mongodb():
    mongo_uri = os.environ.get("MONGODB_URI")
    # Create a new client and connect to the server
    client = MongoClient(mongo_uri)
    # Send a ping to confirm a successful connection
    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)
    
    return client

connect_to_mongodb()

Pinged your deployment. You successfully connected to MongoDB!


MongoClient(host=['ac-ujp5wzn-shard-00-01.h8gtitv.mongodb.net:27017', 'ac-ujp5wzn-shard-00-02.h8gtitv.mongodb.net:27017', 'ac-ujp5wzn-shard-00-00.h8gtitv.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-pvy4a8-shard-0', ssl=True)

In [20]:
print(articles[0])

{'name': "Turkish club ends Israeli footballer's loan spell after Gaza post probe", 'url': 'https://www.msn.com/en-us/news/world/turkish-club-ends-israeli-footballers-loan-spell-after-gaza-post-probe/ar-AA1nbtTZ', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.9Cjgm7cT4Z9wjaRUULHjtS&pid=News', 'width': 700, 'height': 466}}, 'description': 'Turkish first division club Basaksekir on Thursday ended the loan spell of Israeli midfielder Eden Karzev who reposted a social media message calling for the release of hostages held by Hamas militants.', 'about': [{'readLink': 'https://api.bing.microsoft.com/api/v7/entities/fbfb6418-e8cf-0d18-8b81-28d0fcccda7c', 'name': 'Turkey'}, {'readLink': 'https://api.bing.microsoft.com/api/v7/entities/1ffafed3-2b37-b871-c271-aa855d98449a', 'name': 'Israel'}, {'readLink': 'https://api.bing.microsoft.com/api/v7/entities/90eb9ff6-60ad-da43-d3fe-1ee3e3cb775a', 'name': 'Gaza Strip'}], 'mentions': [{'name': 'Turkey'}, {'name': 'Israel'}, {'na